In [0]:
!pip install GetOldTweets3

In [2]:
from google.colab import drive
import pickle
import GetOldTweets3 as got
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Navi Mumbai"
users4=open(DATA_PATH+'/slot3.pickle','rb')
real_tweets3=pickle.load(users4)
users4.close()

print(len(real_tweets3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
21


In [3]:
import queue
crawled=set()
q = queue.Queue(maxsize=2000) 
for i in real_tweets3:      
    q.put(i)
print(q.qsize())
def check_if_rich(q):  
  keywords=['singapore','america','spain','germany','france','paris','berlin','europe','bali','indonesia','switzerland','australia','malaysia','united kingdom','london','mauritius','maldives','thailand','dubai','netherland','tokyo','japan','canada','new york','california','fransisco','angeles','vegas','miami','florida','usa','bermuda','iceland','denmark','luxembourg','kuwait','hong kong','greece','mercedes','bmw','lexus','jaguar','royce','fortuner','audi','bentley','porsche','ferrari','volvo','triumph','enfield','interceptor','rover','cadillac','maserati','lamborghini','hilfiger','rolex','rado','tissot','giordano','diesel','fossil','guess','kors','calvin klein','heuer','armani','adidas','nike','piguet','blanc','hublot','patek', 'bandra', 'khar', 'mahalaxmi', 'south bombay', 'worli', 'vashi', 'colaba','dadar','malabar','lokhandwala','andheri','powai','juhu','peddar','altamount','parel', 'trident','marriot','itc','oberoi','taj','hyatt','lalit','sahara','four seasons','westin','orchid','kohinoor','fariyas','marine plaza','meluha','rodas','radisson','shalimar','waterstones','ambassador','ramada','mirage']
  uname=q.get()
  print(uname)
  while(q.empty()!=True):
    if uname not in crawled:
        crawled.add(uname)
        
        tweetCriteria = got.manager.TweetCriteria().setUsername(uname).setSince("2019-01-01").setUntil("2019-12-31")
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        print("Crawling User: ",uname)
        li=list()
        for j in tweets:
            if any(word in ((j.text).lower()).split() for word in keywords):
                li.append(uname)
                print(li)
                print(j.text)
                break
        print("Going for next user.")
  return li




21


In [0]:
# import queue
# crawled=set()
# q = queue.Queue(maxsize=2000) 
# for i in real_tweets3:      
#     q.put(i)
# print(q.qsize())
def check_if_rich(q):  
  keywords=['singapore','america','spain','germany','france','paris','berlin','europe','bali','indonesia','switzerland','australia','malaysia','united kingdom','london','mauritius','maldives','thailand','dubai','netherland','tokyo','japan','canada','new york','california','fransisco','angeles','vegas','miami','florida','usa','bermuda','iceland','denmark','luxembourg','kuwait','hong kong','greece','mercedes','bmw','lexus','jaguar','royce','fortuner','audi','bentley','porsche','ferrari','volvo','triumph','enfield','interceptor','rover','cadillac','maserati','lamborghini','hilfiger','rolex','rado','tissot','giordano','diesel','fossil','guess','kors','calvin klein','heuer','armani','adidas','nike','piguet','blanc','hublot','patek', 'bandra', 'khar', 'mahalaxmi', 'south bombay', 'worli', 'vashi', 'colaba','dadar','malabar','lokhandwala','andheri','powai','juhu','peddar','altamount','parel', 'trident','marriot','itc','oberoi','taj','hyatt','lalit','sahara','four seasons','westin','orchid','kohinoor','fariyas','marine plaza','meluha','rodas','radisson','shalimar','waterstones','ambassador','ramada','mirage']
  # uname=q.get()
  # print(uname)
  # while(q.empty()!=True):
  #   if uname not in crawled:
  #       crawled.add(uname)
        
        tweetCriteria = got.manager.TweetCriteria().setUsername(uname).setSince("2019-01-01").setUntil("2019-12-31")
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        print("Crawling User: ",uname)
        li=list()
        for j in tweets:
            if any(word in ((j.text).lower()).split() for word in keywords):
                li.append(uname)
                print(li)
                print(j.text)
                break
        print("Going for next user.")
  return li




In [4]:
import concurrent.futures 
import time
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(7) as executor:
    future = executor.submit(check_if_rich, q)
    return_value = future.result()
print("--- %s seconds ---" % (time.time() - start_time))

Madan_R_Darade
Crawling User:  Madan_R_Darade
Going for next user.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-4-daf01be8be47>", line 7, in <module>
    return_value = future.result()
  File "/usr/lib/python3.6/concurrent/futures/_base.py", line 427, in result
    self._condition.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-daf01be8be47>", line 7, in <module>
    return_value = future.result()
  File "/usr/lib/python3.6/concurrent/futures/_base.py", line 611, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.6/concurrent/futures/thread.py", line 152, in shutdown
    t.join()
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  Fi

TypeError: ignored

In [0]:
print(return_value[0])

DreamsQualcon


In [0]:
for i in real_tweets3:
  print(i)

In [14]:
q=queue.Queue(maxsize=5)
if(q.empty()==True):
  print(1)

1


In [11]:
from google.colab import drive
import pickle
import GetOldTweets3 as got
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Navi Mumbai"
users4=open(DATA_PATH+'/slot3.pickle','rb')
real_tweets3=pickle.load(users4)
users4.close()
print(len(real_tweets3))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
21


In [0]:
nino=list()


In [12]:
nino.extend(real_tweets3)

print(len(nino))

133


In [0]:
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Navi Mumbai"
users4=open(DATA_PATH+'/allcommon.pickle','wb')
real_tweets3=pickle.dump(nino,users4)
users4.close()
